In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [2]:
# We will use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 
          'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
          'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 
          'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam',
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 
          'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
          'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California',
          'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

This method will return a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  ). We can see that states have "[edit]" in their names, so, we will use this attribute to distinguish what is a state and what isn't.
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.

In [38]:
def get_list_of_university_towns():    
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


Reading gdplev.xls, we can determinate the year and quarter of the recession start time as a string value in a format such as "2005q3".

In [39]:
def get_recession_start():
    gdp = pd.read_excel('gdplev.xls', skiprows=9, header=None, usecols=[4,5])
    gdp.columns = ['quarter', 'gdp']
    gdp['year'] = gdp['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    gdp = gdp[gdp['year'] >= 2000]
    gdp.reset_index(inplace=True)
    
    for i in range(0, gdp.shape[0]):
        if ((gdp.loc[i,'gdp']) > (gdp.loc[i+1,'gdp']) and
           (gdp.loc[i+1,'gdp']) > (gdp.loc[i+2,'gdp']) and
           (gdp.loc[i+2,'gdp']) < (gdp.loc[i+3,'gdp'])):
            return gdp.loc[i-1, 'quarter']
        
get_recession_start()

'2008q3'

And do the same to get recession end...

In [40]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    gdp = pd.read_excel('gdplev.xls', skiprows=9, header=None, usecols=[4,5])
    gdp.columns = ['quarter', 'gdp']
    gdp['year'] = gdp['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    gdp = gdp[gdp['year'] >= 2000]
    gdp.reset_index(inplace=True)
    
    for i in range(0, gdp.shape[0]):
        if ((gdp.loc[i,'gdp']) > (gdp.loc[i+1,'gdp']) and
           (gdp.loc[i+1,'gdp']) > (gdp.loc[i+2,'gdp']) and
           (gdp.loc[i+2,'gdp']) < (gdp.loc[i+3,'gdp'])):
            return gdp.loc[i+4, 'quarter']
        
get_recession_end()

'2009q4'

As well the recession bottom. Remembering, recession bottom is the quarter that had the lowest GDP during the recession period. To find this, we can verify the start and the end of the recession, with our previously created functions, and apply a `min()` function to get the minimum value. 

In [41]:
def get_recession_bottom():
    gdp = pd.read_excel('gdplev.xls', skiprows=9, header=None, usecols=[4,5])
    gdp.columns = ['quarter', 'gdp']
    gdp['year'] = gdp['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    gdp = gdp[gdp['year'] >= 2000]
    gdp.reset_index(inplace=True)
    
    start = gdp[gdp['quarter'] == get_recession_start()].index[0]
    end = gdp[gdp['quarter'] == get_recession_end()].index[0]
    minGdp = gdp.loc[start:end, 'gdp'].min()
    return gdp.loc[gdp['gdp'] == minGdp, 'quarter'].iloc[0]

get_recession_bottom()

'2009q2'

Reading the ```City_Zhvi_AllHomes.csv```, which contains housing data for the United States, we can see that the information about time is displayed in GDP per month, and not in quarters. So, we need to find the mean for three consecutive months and create a new column for the quarter in the DataFrame. The new columns go for 2000q1 through 2016q3 and should have a multi-index in the shape of ["State", "RegionName"]. We also must replace the values in `State`, which are displayed in acronyms, by the full state name.

In [42]:
def convert_housing_data_to_quarters():
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data = housing_data.drop(housing_data.columns[[0]+list(range(3,51))],axis=1)
    new_housing_data = pd.DataFrame(housing_data[['State','RegionName']])
    
    for year in range(2000,2016):
        new_housing_data[str(year)+'q1'] = housing_data[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        new_housing_data[str(year)+'q2'] = housing_data[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        new_housing_data[str(year)+'q3'] = housing_data[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        new_housing_data[str(year)+'q4'] = housing_data[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)
        
    year = 2016    
    new_housing_data[str(year)+'q1'] = housing_data[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    new_housing_data[str(year)+'q2'] = housing_data[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    new_housing_data[str(year)+'q3'] = housing_data[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    new_housing_data.replace({'State':states}, inplace=True)
    new_housing_data.set_index(['State','RegionName'], inplace=True)
    new_housing_data.sort_index(inplace=True)
    
    return new_housing_data

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

Finally, we reached the core of our investigation: to verify if the recession affects mean house prices in university cities. First, we need to create a column named PriceRatio, got through quarter_before_recession/recession_bottom, this metric reveals us tendencies in housing prices. 

After, we can merge the housing dataframe (containing average sales prices in quarters) with the university towns dataframe (containing the list of university cities). To get the non-university towns, we can get all records of housing dataframe that aren't in university towns dataframe.

### The t-test

With the new column PriceRatio, we can execute the t-test in our dataframes university towns and non university towns, to validate our hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom.

We defined the p-value as 0.01, which is statistically significant. P-value is the level of marginal significance within a statistical hypothesis test, representing the probability of the occurrence of a given event.

If the t-test returns p < 0.01, we reject the null hypothesis. Otherwise, we accept the null hypothesis.

In [50]:
def run_ttest():
    unitowns = get_list_of_university_towns()
    bottom_recession = get_recession_bottom()
    start_recession = get_recession_start()
    housing_data = convert_housing_data_to_quarters()
    before_start_recession = housing_data.columns[housing_data.columns.get_loc(start_recession) -1]
    
    housing_data['PriceRatio'] = housing_data[before_start_recession] / housing_data[bottom_recession]
    housing_data = housing_data[[bottom_recession, before_start_recession, 'PriceRatio']]
    housing_data = housing_data.reset_index()
    unitowns_hdata = pd.merge(housing_data,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    
    new_housing_data = pd.merge(housing_data,unitowns_hdata,how='outer',on=['State','RegionName', bottom_recession, before_start_recession,'PriceRatio'])
    new_housing_data['uni'] = new_housing_data['uni'].fillna(False)

    university_towns = new_housing_data[new_housing_data['uni'] == True]
    non_university_towns = new_housing_data[new_housing_data['uni'] == False]

    t,p = ttest_ind(university_towns['PriceRatio'].dropna(),non_university_towns['PriceRatio'].dropna())
    
    different = True if p < 0.01 else False

    better = "non-university town" if university_towns['PriceRatio'].mean() < non_university_towns['PriceRatio'].mean() else "university town"

    return different, p, better

run_ttest()

(True, 0.002724063704761164, 'non-university town')

### Conclusion
As our t-test returned p < 0.01, therefore, the alternative hypothesis is accepted and it is proven that the average housing prices in university cities are, in fact, less affected during recessions.